In [2]:
#!python -m pip install featuretools

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 522.5/522.5 kB 29.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 230.5/230.5 kB 37.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 kB 35.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 930.1/930.1 kB 64.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 51.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.5/139.5 kB 33.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.8/55.8 kB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.9/47.9 kB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 83.3 MB/s eta 0:00:00ta 0:00:01
  Preparing metadata (setup.py) ... done
  Created wheel for PyMeeus: filename=PyMeeus-0.5.12-py3-none-any.whl size=731999 sha256=90c97ed91cd8881345cf22ef9dc7af1785732ff22eb9f12e2014a6500706dd06
  Stored in directory: /opt/ml/.cache/pip/wheels/b9/49/19/1ec4bcf2cc76

In [2]:
import torch
import pandas as pd
import numpy as np
# pip install transformers
from transformers import BertTokenizer, BertModel
# 만약 주피터 노트북에서 아래와 관계있는 에러가 발생한다면
# pip install ipywidgets

In [3]:
main_df = pd.read_csv('/opt/ml/input/data/train/train_ratings.csv') # user-item-time
title_df = pd.read_csv('/opt/ml/input/data/train/titles.tsv', sep='\t') # item-title
year_df = pd.read_csv('/opt/ml/input/data/train/years.tsv', sep='\t') # item-year
director_df = pd.read_csv('/opt/ml/input/data/train/directors.tsv', sep='\t') # item-director
genre_df = pd.read_csv('/opt/ml/input/data/train/genres.tsv', sep='\t') # item-genre(name)
writer_df = pd.read_csv('/opt/ml/input/data/train/writers.tsv', sep='\t') # item-writer

### writer별 가장 평점을 가장 많이 받은 영화의 평점수(wri_max_cnt)와, 기여했던 모든 영화의 총합 평점수(wri_sum_cnt)

In [4]:
# director 결측 item 작가를 'nm0000000'로 채워주기
no_wri_item_list = list(set(main_df['item']) - set(writer_df['item']))
no_wri_item_df = pd.DataFrame([x for x in zip(no_wri_item_list, ['nm0000000']*len(no_wri_item_list))])
no_wri_item_df.columns=writer_df.columns # 컬럼명 동일하게
writer_df = pd.concat([writer_df, no_wri_item_df]) # 기존 writer_df 뒤에 작가없는 영화 추가

In [6]:
writer_df['writer'].nunique()

2990

In [7]:
item_popularity = pd.read_csv('/opt/ml/input/fighting/FE/item/item_interaction_cnt.csv')

In [8]:
item_popularity.head(3) #6807 row

,item,item_cnt
0,2571,19699
1,2959,18437
2,296,18202


In [9]:
writer_df.merge(item_popularity, how='left', on='item')

,item,writer,item_cnt
0,1237,nm0000005,1383
1,5147,nm0000005,949
2,7327,nm0000005,751
3,2068,nm0000005,502
4,7396,nm0000005,335
...,...,...,...
12460,55284,nm0000000,207
12461,81910,nm0000000,56
12462,4091,nm0000000,204
12463,2046,nm0000000,845


In [10]:
sum(writer_df[writer_df['writer']=='nm0000005'].merge(item_popularity, how='left', on='item')['item_cnt'].values)


6020

In [12]:
writer_list = list(writer_df['writer'].unique())

In [13]:
result = []
for writer in writer_list:
    max_cnt = max(writer_df[writer_df['writer']==writer].merge(item_popularity, how='left', on='item')['item_cnt'].values) # 해당 작가가 가장 많이 받은 interaction의 수
    sum_cnt = sum(writer_df[writer_df['writer']==writer].merge(item_popularity, how='left', on='item')['item_cnt'].values) # 해당 작가가 받은 interaction의 누적수
    result.append((writer, max_cnt, sum_cnt))
result_df = pd.DataFrame(result)

In [14]:
result_df.columns = ['writer','wri_max_cnt','wri_sum_cnt']
result_df

,writer,wri_max_cnt,wri_sum_cnt
0,nm0000005,1383,6020
1,nm0000019,905,3531
2,nm0000033,1190,1787
3,nm0000036,813,1574
4,nm0000040,9155,48618
...,...,...,...
2985,nm5335213,821,1107
2986,nm5371819,796,1910
2987,nm5927607,5118,5171
2988,nm5927608,5118,5171


In [15]:
### 'nm0000000'의 경우는 작가정보가 안주어진 경우이다. 따라서 중간값을 취해준다.

In [16]:
result_df['wri_max_cnt'].describe()

count     2990.000000
mean      2056.258194
std       2951.967389
min         45.000000
25%        283.000000
50%        816.500000
75%       2523.000000
max      19699.000000
Name: wri_max_cnt, dtype: float64

In [17]:
result_df.loc[result_df['writer'] == 'nm0000000', 'wri_max_cnt'] = 816
result_df.loc[result_df['writer'] == 'nm0000000', 'wri_sum_cnt'] = 816

In [18]:
result_df

,writer,wri_max_cnt,wri_sum_cnt
0,nm0000005,1383,6020
1,nm0000019,905,3531
2,nm0000033,1190,1787
3,nm0000036,813,1574
4,nm0000040,9155,48618
...,...,...,...
2985,nm5335213,821,1107
2986,nm5371819,796,1910
2987,nm5927607,5118,5171
2988,nm5927608,5118,5171


In [19]:
result_df.to_csv("writer_max_sum_intercnt.csv", index=False)